프로그램이 구체적으로 처리해야 하는 정보의 유형이 무엇이든, 리스트에서 index 함수를 사용해 특정 값을 찾아내려면 리스트 길이에 선형으로 비례하는 시간이 필요하다

In [1]:
data = list(range(10**5))
index = data.index(91234)
assert index == 91234

In [2]:
def find_closest(sequence, goal):
    for index, value in enumerate(sequence):
        if goal < value:
            return index
    
    raise ValueError(f'범위를 벗어남: {goal}')

index = find_closest(data, 91234.56)
assert index == 91235

파이썬 내장 bisect 모듈은 순서가 정해져 있는 리스트에 대해 이런 유형의 검사를 더 효과적으로 수행한다.

bisect_left 함수를 사용하면 정렬된 원소로 이루어진 시퀸스에 대해 이진 검색을 효율적으로 수행 할 수 있다.

bisect_left가 반환하는 인덱스는 리스트에 찾는 값의 원소가 존재하지 않는 경우 이 원소의 인덱스 이며

리스ㅡㅌ에 찾는 값의 원소가 존재하지 않을 경우 정렬 순서상 해당 값을 삽입해야할 자리의 인덱스다

In [3]:
from bisect import bisect_left

index = bisect_left(data, 91234) # 정확히 일치
assert index == 91234

index = bisect_left(data, 91234.56) # 근접한 값과 일치
assert index == 91235

index = bisect_left(data, 91234.33) # 근접한 값과 일치 ( 찾는 값 이상의 값 중에서 근접한 값을 찾음 )
assert index == 91235


timeit을 사용해 마이크로 벤치마크를 수행해서 bisect를 사용하면 실제 성능이 향상되는지 검증 할 수 있다.

In [4]:
import random
import timeit

size = 10 ** 5
iterations = 1000

data = list(range(size))
to_lookup = [random.randint(0, size) for _ in range(iterations)]

def run_linear(data, to_lookup):
    for index in to_lookup:
        data.index(index)
        
def run_bisect(data, to_lookup):
    for index in to_lookup:
        bisect_left(data, index)
        
        
baseline = timeit.timeit(
    stmt= 'run_linear(data, to_lookup)',
    globals = globals(),
    number = 10
)
print(f'선형 검색: {baseline :.6f} 초')

comparison = timeit.timeit(
    stmt = 'run_bisect(data, to_lookup)',
    globals=globals(),
    number=10
)
print(f'이진 검색: {comparison: .6f} 초')


slowdown = 1 + ((baseline -comparison) / comparison)

print(f'선형 검색이 {slowdown:.1f}배 더걸림')

선형 검색: 15.796686 초
이진 검색:  0.017057 초
선형 검색이 926.1배 더걸림


bisect에서 가장 좋은 점은 리스트 타입 뿐만 아니라 시퀀스 처럼 작동

모든 파이썬 객체에 대해 bisect 모듈의 기능을 사용할 수 있다는 점이다.